## Lab Session 

### Learning Objective:
- Working with data using python libaries.
- Data Visualization.
- Exploratory data analysis and data preprocessing.
- Building a Linear regression model to predict the tip amount based on different input features.

### About the dataset (Customer Tip Data)

#### Dataset Source: https://www.kaggle.com/datasets/ranjeetjain3/seaborn-tips-dataset

The dataset contains information about the 244 orders served at a restaurant in the United States. Each observation includes the factors related to the order like total bill, time, the total number of people in a group, gender of the person paying for the order and so on.

#### Attribute Information:

- **total_bill:** Total bill (cost of the meal), including tax, in US dollars
- **tip:** Tip in US dollars
- **sex:** Sex of person paying for the meal
- **smoker:** There is a smoker in a group or not
- **day:** Day on which the order is served
- **time:** Time of the order
- **size:** Size of the group

Food servers’ tips in restaurants may be influenced by many factors, including the nature of the restaurant, size of the party, and table locations in the restaurant. Restaurant managers need to know which factors matter when they assign tables to food servers. For the sake of staff morale, they usually want to avoid either the substance or the appearance of unfair
treatment of the servers, for whom tips (at least in restaurants in the UnitedStates) are a major component of pay.

### Load the dataset

In [ ]:
url = "https://www.kaggle.com/datasets/ranjeetjain3/seaborn-tips-dataset"
tips_data = pd.read_csv(url)

### Import required libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

### 1. Make a list of categorical and numerical columns in the data.

In [ ]:
categorical_columns = tips_data.select_dtypes(include='object').columns.tolist()
numerical_columns = tips_data.select_dtypes(exclude='object').columns.tolist()


### 2. Compute the average bill amount for each day.

In [ ]:
average_bill_per_day = tips_data.groupby('day')['total_bill'].mean()

### 3. Which gender is more generous in giving tips?

In [ ]:
average_tip_by_gender = tips_data.groupby('sex')['tip'].mean()
more_generous_gender = average_tip_by_gender.idxmax()


### 4. According to the data, were there more customers for dinner or lunch?

In [ ]:
order_counts_by_time = tips_data['time'].value_counts()
more_customers_time = order_counts_by_time.idxmax()

### 5. Based on the statistical summary, comment on the variable 'tip'

In [ ]:
tip_summary = tips_data['tip'].describe()

### 6. Find the busiest day in terms of the orders?

In [ ]:
busiest_day = tips_data['day'].value_counts().idxmax()

### 7. Is the variable 'total_bill' skewed? If yes, identify the type of skewness. Support your answer with a plot

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(tips_data['total_bill'], kde=True)
plt.title('Distribution of Total Bill Amount')
plt.xlabel('Total Bill Amount')
plt.ylabel('Frequency')
plt.show()
total_bill_skewness = tips_data['total_bill'].skew()

### 8. Is the tip amount dependent on the total bill? Visualize the relationship with a appropriate plot and metric and write your findings.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_bill', y='tip', data=tips_data, hue='sex', palette='viridis')
plt.title('Scatter Plot of Total Bill vs. Tip')
plt.xlabel('Total Bill ($)')
plt.ylabel('Tip ($)')
plt.show()
tip_total_bill_correlation = tips_data[['total_bill', 'tip']].corr().iloc[0, 1]

### 9. What is the percentage of males and females in the dataset? and display it in the plot

In [ ]:
gender_percentage = tips_data['sex'].value_counts(normalize=True) * 100
plt.figure(figsize=(6, 4))
sns.countplot(x='sex', data=tips_data)
plt.title('Gender Distribution')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

### 10. Compute the gender-wise count based on smoking habits and display it in the plot

In [ ]:
gender_smoker_count = tips_data.groupby(['sex', 'smoker']).size().unstack()
gender_smoker_count.plot(kind='bar', stacked=True)
plt.title('Gender-wise Count Based on Smoking Habits')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

### 11. Compute the average tip amount given for different days and display it in the plot.

In [ ]:
average_tip_by_day = tips_data.groupby('day')['tip'].mean()
plt.figure(figsize=(8, 6))
sns.barplot(x=average_tip_by_day.index, y=average_tip_by_day.values)
plt.title('Average Tip Amount by Day')
plt.xlabel('Day')
plt.ylabel('Average Tip Amount')
plt.show()

### 12. Is the average bill amount dependent on the size of the group? Visualize the relationship using appropriate plot and write your findings.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='size', y='total_bill', data=tips_data)
plt.title('Scatter Plot of Size vs. Total Bill')
plt.xlabel('Size of the Group')
plt.ylabel('Total Bill ($)')
plt.show()
bill_size_correlation = tips_data[['size', 'total_bill']].corr().iloc[0, 1]

### 13. Plot a horizontal boxplot to compare the bill amount based on gender

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='total_bill', y='sex', data=tips_data)
plt.title('Boxplot of Total Bill Amount by Gender')
plt.xlabel('Total Bill ($)')
plt.ylabel('Gender')
plt.show()

### 14. Find the maximum bill amount for lunch and dinner on Saturday and Sunday

In [ ]:
max_bill_lunch = tips_data.loc[(tips_data['day'].isin(['Sat', 'Sun'])) & (tips_data['time'] == 'Lunch'), 'total_bill'].max()
max_bill_dinner = tips_data.loc[(tips_data['day'].isin(['Sat', 'Sun'])) & (tips_data['time'] == 'Dinner'), 'total_bill'].max()


### 15. Compute the percentage of missing values in the dataset.

In [ ]:
percentage_missing_values = (tips_data.isnull().sum() / len(tips_data)) * 100


### 16. Is there are any duplicate records in the dataset? If yes compute the count of the duplicate records and drop them.

In [ ]:
duplicate_count = tips_data.duplicated().sum()
tips_data_no_duplicates = tips_data.drop_duplicates()


### 17. Is there are any outliers present in the column 'total_bill'? If yes treat them with transformation approach, and plot a boxplot before and after the treatment

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='total_bill', data=tips_data)
plt.title('Boxplot of Total Bill Amount Before Outlier Treatment')
plt.xlabel('Total Bill ($)')
plt.show()


### 18. Is there are any outliers present in the column 'tip'? If yes remove them using IQR techinque.

In [ ]:
Q1 = tips_data['tip'].quantile(0.25)
Q3 = tips_data['tip'].quantile(0.75)
IQR = Q3 - Q1
tips_data_no_outliers = tips_data[(tips_data['tip'] >= Q1 - 1.5 * IQR) & (tips_data['tip'] <= Q3 + 1.5 * IQR)]


### 19. Encode the categorical columns in the dataset and print the random 5 samples from the dataframe.

In [ ]:
tips_data_encoded = pd.get_dummies(tips_data, columns=categorical_columns, drop_first=True)
print(tips_data_encoded.sam

### 20. Check the range of the column 'total_bill' and transform the values such that the range will be 1.

In [ ]:
total_bill_range = tips_data['total_bill'].max() - tips_data['total_bill'].min()
tips_data['total_bill_transformed'] = (tips_data['total_bill'] - tips_data['total_bill'].min()) / total_bill_range


### 21. Load the dataset again by giving the name of the dataframe as "tips_df"
- i) Encode the categorical variables.
- ii) Store the target column (i.e.tip) in the y variable and the rest of the columns in the X variable

In [ ]:
tips_df = pd.read_csv(url)
# i) Encode the categorical variables.
tips_df_encoded = pd.get_dummies(tips_df, columns=categorical_columns, drop_first=True)

# ii) Store the target column (i.e. tip) in the y variable and the rest of the columns in the X variable
X = tips_df_encoded.drop('tip', axis=1)
y = tips_df_encoded['tip']


### 22. Split the dataset into two parts (i.e. 70% train and 30% test), and Standardize the columns "total_bill" and "Size" using the mim_max scaling approach

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = MinMaxScaler()
X_train[['total_bill', 'size']] = scaler.fit_transform(X_train[['total_bill', 'size']])
X_test[['total_bill', 'size']] = scaler.transform(X_test[['total_bill', 'size']])


### 23. Train a linear regression model using the training data and print the r_squared value of the prediction on the test data.

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
r2_value = r2_score(y_test, y_pred)

print(f"R-squared value on the test data: {r2_value}")

### Happy Learning:)